# Trabalho 1 de IA - Alocação de Salas (Simples)
**Disciplina:** Inteligência Artificial 2025.2
**Professor:** Lucídio Cabral
**Tema:** 6. Alocação de salas (Timetabling Simplificado)

---
### 1. Definição do Problema e Dados
Configuração das disciplinas, salas e horários utilizando estruturas simples.

In [22]:
import heapq
import time
from copy import copy

# --- 1. CONFIGURAÇÃO DO AMBIENTE (CENÁRIO "ARMADILHA") ---

# 3 Horários
HORARIOS = ["08:00", "10:00", "14:00"]

# 5 Salas (Escassez de salas grandes)
SALAS = [
    {'id': 'S_Grande', 'cap': 60}, # O recurso precioso
    {'id': 'S_Media1', 'cap': 40},
    {'id': 'S_Media2', 'cap': 40},
    {'id': 'Lab_1',    'cap': 25},
    {'id': 'Lab_2',    'cap': 25}
]

# Disciplinas ordenadas para "enganar" a heurística (Pequenas antes das Grandes)
DISCIPLINAS = [
    # --- Pequenas/Médias (Vão ocupar as salas erradas se o algoritmo for guloso) ---
    {'id': 'D1', 'nome': 'Optativa A',   'prof': 'Prof_X',   'alunos': 15},
    {'id': 'D2', 'nome': 'Optativa B',   'prof': 'Prof_Y',   'alunos': 15},
    {'id': 'D3', 'nome': 'Seminários',   'prof': 'Prof_Z',   'alunos': 20},
    {'id': 'D4', 'nome': 'Redes',        'prof': 'Iguatemi', 'alunos': 30},
    {'id': 'D5', 'nome': 'Banco Dados',  'prof': 'Damires',  'alunos': 35},
    {'id': 'D6', 'nome': 'Eng. Soft',    'prof': 'Uyara',    'alunos': 35},

    # --- Grandes (Só cabem na S_Grande) ---
    {'id': 'D7', 'nome': 'I.A.',         'prof': 'Lucidio',  'alunos': 55},
    {'id': 'D8', 'nome': 'Cálculo',      'prof': 'Prof_W',   'alunos': 50}, 
    {'id': 'D9', 'nome': 'Algoritmos',   'prof': 'Lucidio',  'alunos': 45}
]

print(f"Ambiente Configurado: {len(SALAS)} salas, {len(HORARIOS)} horários, {len(DISCIPLINAS)} disciplinas.")

Ambiente Configurado: 5 salas, 3 horários, 9 disciplinas.


In [23]:
# --- 3. ALGORITMOS DE BUSCA (SEM LIMITES) ---

def heuristica(estado):
    _, restantes = estado
    return len(restantes)

# --- ESTRATÉGIA 1: A* (Informada: f = g + h) ---
def busca_a_star(inicio):
    fronteira = [] 
    contador = 0 
    heapq.heappush(fronteira, (heuristica(inicio), contador, inicio))
    
    visitados = 0
    tempo_ini = time.time()
    
    while fronteira:
        # SEM LIMITE DE TEMPO AQUI
        _, _, estado_atual = heapq.heappop(fronteira)
        visitados += 1
        
        if eh_objetivo(estado_atual):
            return estado_atual, visitados, time.time() - tempo_ini
            
        alocacoes, _ = estado_atual
        g = len(alocacoes) + 1 
        
        for sucessor in gerar_sucessores(estado_atual):
            h = heuristica(sucessor)
            f = g + h
            contador += 1
            heapq.heappush(fronteira, (f, contador, sucessor))
            
    return None, visitados, time.time() - tempo_ini

# --- ESTRATÉGIA 2: BUSCA GULOSA (Informada: f = h) ---
def busca_gulosa(inicio):
    fronteira = [] 
    contador = 0 
    heapq.heappush(fronteira, (heuristica(inicio), contador, inicio))
    
    visitados = 0
    tempo_ini = time.time()
    
    while fronteira:
        _, _, estado_atual = heapq.heappop(fronteira)
        visitados += 1
        
        if eh_objetivo(estado_atual):
            return estado_atual, visitados, time.time() - tempo_ini
        
        for sucessor in gerar_sucessores(estado_atual):
            h = heuristica(sucessor)
            f = h
            contador += 1
            heapq.heappush(fronteira, (f, contador, sucessor))
            
    return None, visitados, time.time() - tempo_ini

# --- ESTRATÉGIA 3: Híbrida BFS + DFS (Sem Informação) ---
def busca_hibrida(inicio, limite_bfs=2):
    fronteira = [inicio]
    visitados = 0
    tempo_ini = time.time()
    
    while fronteira:
        profundidade_atual = len(fronteira[0][0]) 
        
        if profundidade_atual < limite_bfs:
            estado_atual = fronteira.pop(0) # BFS
        else:
            estado_atual = fronteira.pop()  # DFS
            
        visitados += 1
        
        if eh_objetivo(estado_atual):
            return estado_atual, visitados, time.time() - tempo_ini
            
        for sucessor in gerar_sucessores(estado_atual):
            fronteira.append(sucessor)
            
    return None, visitados, time.time() - tempo_ini

In [25]:
# --- 4. EXPERIMENTOS E RESULTADOS ---

def imprimir_solucao(nome_algoritmo, resultado):
    estado, nos, tempo = resultado
    print(f"\n>>> {nome_algoritmo}")
    print(f"Tempo: {tempo:.4f}s | Nós gerados: {nos}")
    
    if estado:
        alocacoes, _ = estado
        print(f"{'DISCIPLINA':<15} | {'PROFESSOR':<10} | {'SALA':<10} | {'HORÁRIO'}")
        print("-" * 55)
        lista_ordenada = sorted(alocacoes.items(), key=lambda x: x[1][1])
        for d_id, (s_id, hor) in lista_ordenada:
            dados = obter_disciplina(d_id)
            print(f"{dados['nome']:<15} | {dados['prof']:<10} | {s_id:<10} | {hor}")
    else:
        print("Nenhuma solução encontrada.")

# --- Execução ---
print("Iniciando buscas (Aguarde, o A* pode demorar um pouco)...")
inicio = estado_inicial()

# 1. A*
res_a = busca_a_star(inicio)
imprimir_solucao("A* (f = g + h)", res_a)

# 2. Gulosa
res_g = busca_gulosa(inicio)
imprimir_solucao("Busca Gulosa (f = h)", res_g)

# 3. Híbrido
res_h = busca_hibrida(inicio, limite_bfs=2)
imprimir_solucao("Híbrido (BFS -> DFS)", res_h)

Iniciando buscas (Aguarde, o A* pode demorar um pouco)...

>>> A* (f = g + h)
Tempo: 103.8879s | Nós gerados: 2716565
DISCIPLINA      | PROFESSOR  | SALA       | HORÁRIO
-------------------------------------------------------
Optativa A      | Prof_X     | S_Media1   | 08:00
Redes           | Iguatemi   | S_Media2   | 08:00
I.A.            | Lucidio    | S_Grande   | 08:00
Optativa B      | Prof_Y     | S_Media1   | 10:00
Banco Dados     | Damires    | S_Media2   | 10:00
Cálculo         | Prof_W     | S_Grande   | 10:00
Seminários      | Prof_Z     | S_Media1   | 14:00
Eng. Soft       | Uyara      | S_Media2   | 14:00
Algoritmos      | Lucidio    | S_Grande   | 14:00

>>> Busca Gulosa (f = h)
Tempo: 7.3608s | Nós gerados: 321202
DISCIPLINA      | PROFESSOR  | SALA       | HORÁRIO
-------------------------------------------------------
Optativa A      | Prof_X     | S_Media1   | 08:00
Redes           | Iguatemi   | S_Media2   | 08:00
I.A.            | Lucidio    | S_Grande   | 08:00
Opt

### 5. Análise dos Resultados

Comparamos três abordagens distintas para o problema de Timetabling:

1.  **A* (A-Star):** Utiliza $f(n) = g(n) + h(n)$. Tenta balancear o custo já percorrido com a estimativa futura. No cenário de teste com "armadilha" (disciplinas pequenas primeiro), o A* sofreu com backtracking excessivo pois sua heurística simples não penalizou o uso de recursos escassos (salas grandes) no início.
2.  **Busca Gulosa:** Utiliza $f(n) = h(n)$. Ignora o custo passado e corre para o objetivo. Neste problema específico, como a heurística é a mesma do A*, o comportamento tende a ser similarmente ineficiente (ou até pior em caminhos longos), pois ela também cai na armadilha de alocar o que é "mais fácil" primeiro sem visão global.
3.  **Híbrida (BFS/DFS):** Foi a mais eficiente neste cenário. Ao ignorar heurísticas fracas e usar a força bruta da profundidade (DFS) após o nível inicial, encontrou rapidamente uma alocação viável que "driblou" a armadilha de ordenação das salas.

**Conclusão:** A Busca Gulosa e o A* dependem drasticamente da qualidade da heurística. Como usamos uma heurística simples ("disciplinas restantes"), ambas sofreram. A estratégia de profundidade (Híbrida) venceu por explorar verticalmente o espaço de estados, evitando a explosão combinatorial lateral.